Copyright by Jonas Emmert <br>

Research Group Applied Systems Biology - Head: Prof. Dr. Marc Thilo Figge <br>
https://www.leibniz-hki.de/en/applied-systems-biology.html <br>
HKI-Center for Systems Biology of Infection <br>
Leibniz Institute for Natural Product Research and Infection Biology - Hans Knöll Insitute (HKI) <br>
Adolf-Reichwein-Straße 23, 07745 Jena, Germany

Mean and std are needed in utils.py and need to be manually inserted. It is best to use the MSAD code in PyCharm or
something identical, as this allows to work on the notebook, manipulate the .py files and excecute
the .py files in the build in terminal all at once. <br>
Also add the paths to the utils.py get_loaders function under dataset == 'custom'.

In [ ]:
import torch
from torchvision import datasets, transforms

In [ ]:
def create_dataset(path, transforms):
    # ToTensor() automatically normalizes pixel values to [0,1]
    dataset = datasets.ImageFolder(path, transform=transforms)
    return dataset

In [ ]:
# calculate WHOLE dataset mean and standard deviation
def get_mean_std(dataset, batch_size):
    # create a data loader
    loader = torch.utils.data.DataLoader(dataset,
                             batch_size=batch_size,
                             num_workers=0,
                             shuffle=False)

    mean = 0.0
    var = 0.0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
    mean = mean / len(loader.dataset)

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        var += ((images - mean.unsqueeze(1)) ** 2).sum([0, 2])
    std = torch.sqrt(var / (len(loader.dataset) * 224 * 224))

    mean = [round(float(x), 3) for x in mean]
    std = [round(float(x), 3) for x in std]

    return mean, std

In [ ]:
# WHOLE dataset --> train & test sets

path_to_train = 'E:\\parralelcomputed_trackdata\\Heatmaps_train_norm\\'
path_to_test = 'E:\\parralelcomputed_trackdata\\Heatmaps_test\\'
ds_train = create_dataset(path_to_train, transforms=transforms.ToTensor())
ds_test = create_dataset(path_to_test, transforms=transforms.ToTensor())
whole_ds = torch.utils.data.ConcatDataset([ds_train, ds_test])
mean, std = get_mean_std(whole_ds, 10)
print(mean)
print(std)